导入必要的包

In [1]:
import numpy as np
import yaml
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
import sys

In [2]:
sys.path.append('opt/conda/envs/PointNet2.1/')
sys.path.append('/kitti_semantic/Pointnet2_PyTorch-master')

In [3]:
from pointnet2.models import Pointnet2SemMSG as Pointnet
from pointnet2.models.pointnet2_msg_sem import model_fn_decorator
from pointnet2.data.KittiLoader import Kitti3DSemSeg

import pointnet2.utils.pytorch_utils as pt_uitls

设置参数

In [4]:
# 预测结果输出文件
preds_path = '/kitti_semantic/dataset/sequences/00/preds1'
# batch_size
batch_size = 1
# num_points
num_points = 100592
# device_name
device_name = 'cuda'

In [5]:
# 学习率
lr_clip = 1e-5
bnm_clip = 1e-2

读取测试数据

In [6]:
test_set = Kitti3DSemSeg(num_points, train = False)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle = False, pin_memory = True, num_workers = 0)

34
(10059200, 3)
9799219
(100, 100592, 9)
(100, 100592)
19


设置模型

In [7]:
device = torch.device(device_name)
model = Pointnet(num_classes = 34, input_channels = 6, use_xyz = True)
model.load_state_dict(torch.load(
    '/kitti_semantic/Pointnet2_PyTorch-master/pointnet2/train/pointnet2_semseg_best.pth.tar', 
    map_location = device
)['model_state'])
model.cuda()

Pointnet2MSG(
  (SA_modules): ModuleList(
    (0): PointnetSAModuleMSG(
      (groupers): ModuleList(
        (0): QueryAndGroup()
        (1): QueryAndGroup()
      )
      (mlps): ModuleList(
        (0): SharedMLP(
          (layer0): Conv2d(
            (conv): Conv2d(9, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer1): Conv2d(
            (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer2): Conv2d(
            (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(
              (bn): BatchNor

In [16]:
# 读取remap
config_path = '/kitti_semantic/Pointnet2_PyTorch-master/pointnet2/data/kitti_api/config/semantic-kitti.yaml'
CFG = yaml.safe_load(open(config_path, 'r'))
class_inv_map = CFG['learning_map_inv']

label_path = '/kitti_semantic/dataset/sequences/00/labels'

for i,data in enumerate(test_loader):
    # f = open(os.path.join(preds_path, str(i).zfill(6) + '.label'), 'w')
    f = os.path.join(preds_path, str(i).zfill(6) + '.label')
    f_l = os.path.join(label_path, str(i).zfill(6) + '.label')
    inputs, labels = data
    inputs = inputs.to(device, non_blocking = True)
    labels = labels.to(device, non_blocking = True)
    
    preds = model(inputs)
    _, classes = torch.max(preds, -1)

    acc = (classes == labels).float().sum() / labels.numel()
    print(acc.item())

    classes = classes.clone().cpu().numpy()
    classes = classes.reshape(num_points).tolist()
    classes_bin = []
    for i in range(len(classes)):
        classes_bin.append(class_inv_map[classes[i]])

    # 找出与源点云的数量的差值
    k = len(np.fromfile(f_l,dtype = np.uint32).tolist()) - len(classes)
    for i in range(k):
        classes_bin.append(0)

    classes_bin = np.array(classes_bin, dtype = np.uint32)
    print(classes_bin.shape)
    classes_bin.tofile(f)

    # for n in range(num_points):
    #     f.write(str(classes_bin[n]) + '\n')
    # f.close()

    # print('\n')
    # print(classes)

0.6846567988395691
(124668,)
0.6574081182479858
(124605,)
0.73968106508255
(124478,)
0.7498210668563843
(124167,)
0.8148460984230042
(123969,)
0.8679516911506653
(123924,)
0.8787179589271545
(123373,)
0.8586269021034241
(122765,)
0.8545907735824585
(122123,)
0.8127882480621338
(121357,)
0.8788471817970276
(121018,)
0.8570959568023682
(120945,)
0.8903490900993347
(120503,)
0.9179656505584717
(120295,)
0.8698703646659851
(120406,)
0.9196456670761108
(121023,)
0.9018311500549316
(121842,)
0.9235624670982361
(123107,)
0.8910449743270874
(123940,)
0.8254930377006531
(124301,)
0.8084738254547119
(124553,)
0.7850027680397034
(124656,)
0.7372554540634155
(124644,)
0.8191108703613281
(124526,)
0.8631998300552368
(124520,)
0.9041076898574829
(124600,)
0.9126272201538086
(124550,)
0.8748707175254822
(124242,)
0.8917607665061951
(124163,)
0.9229461550712585
(124018,)
0.9090285301208496
(124134,)
0.8995347619056702
(124140,)
0.9219719171524048
(123932,)
0.9329668283462524
(123826,)
0.92477530241012

In [11]:
torch.cuda.is_available()

True

二进制

In [17]:
l = 12
i = 3

label_demo = np.fromfile('/kitti_semantic/dataset/sequences/00/preds1/000000.label', dtype = np.uint32)
print(label_demo.shape)


(124668,)


In [14]:
print((label_demo >> 16)[0 : 100])
print((label_demo & 0xFFFF)[0 : 100])
print(label_demo[0 : 100])


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 71 71 71 71 71 71 71 71  0  0 71  0  0 71  0  0  0 71 71
 71 71  0 71 71 71 71 71 81 81 50 50 50 50 50 50 50 80 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 71 71 71 71 71 71 71 71  0  0 71  0  0 71  0  0  0 71 71
 71 71  0 71 71 71 71 71 81 81 50 50 50 50 50 50 50 80 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50]


In [15]:
l = np.array([71], dtype = np.uint32)

l_tmp = l[0]

print(l_tmp & 0xFFFF)
print(l_tmp >> 16)



71
0
